# Direct API Access

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ionq-samples/getting-started/blob/main/api.ipynb)

IonQ offers a set of API resources that allow you to submit and manage jobs directly. This notebook offers a few simple examples, but visit [docs.ionq.com](https://docs.ionq.com) for a complete guide with additional resources and functionality.

In [ ]:
import json
import time

import requests

from helpers import get_ionq_api_key

# Before you begin, get your API key from https://cloud.ionq.com/settings/keys

# If your API key is stored as "IONQ_API_KEY" in your local environment, this
# should find it. Otherwise you'll be prompted to enter your API key manually.

api_key = get_ionq_api_key()

In [ ]:
# Define helper functions for interacting with IonQ's API.

def submit_job(headers, data):
    """Submit a quantum job to IonQ's API."""
    url = "https://api.ionq.co/v0.3/jobs"
    response = requests.post(url, headers=headers, data=data)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["id"]

def query_job(job_id, headers):
    """Query the status of a submitted job."""
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json["status"]

def get_job_results(job_id, headers):
    """Retrieve the results of a completed job."""
    url = f"https://api.ionq.co/v0.3/jobs/{job_id}/results"
    response = requests.get(url, headers=headers)
    response_json = response.json()
    assert response.status_code == 200, f"Error: {response_json.get('message', 'Unknown error')}"
    return response_json

In [ ]:
# Set up the request headers and define our circuit.
# This circuit creates a Bell state with two qubits using IonQ's native format.

headers = {
    "Authorization": f"apiKey {api_key}",
    "Content-Type": "application/json",
}

data = {
    "name": "API Example Circuit",
    "shots": 100,
    "target": "simulator",
    "input": {
        "format": "ionq.circuit.v0",
        "gateset": "qis",
        "qubits": 2,
        "circuit": [
            {
                "gate": "h",
                "target": 0
            },
            {
                "gate": "cnot",
                "control": 0,
                "target": 1
            }
        ]
    }
}


```json
{
    "shots": 100,
    "target": "simulator",
    "input": {
        "format": "ionq.circuit.v0",
        "gateset": "qis",
        "qubits": 2,
        "circuit": [
            {
                "gate": "h",
                "target": 0
            },
            {
                "gate": "cnot",
                "control": 0,
                "target": 1
            }
        ]
    }
}
```

In [ ]:
# Now we'll send the job to our backend for processing.

job_id = submit_job(headers, json.dumps(data))

# And wait for the job to complete.

status = "ready"
while status != "completed":
    time.sleep(1)  # Wait for 1 second before querying again.
    status = query_job(job_id, headers)

# Once the job has completed, we can retrieve and display the results.

results = get_job_results(job_id, headers)
print(results)

## And that's a wrap!

To explore more of IonQ's APIs, check out the complete docs at https://docs.ionq.com/